In [ ]:
import pandas as pd
import numpy as np
import json
import os
from typing import Dict, Any, List
import matplotlib.pyplot as plt

**read the metrics**

In [ ]:
import dotenv

dotenv.load_dotenv(override=True)

CONFIG_FILE_PATH = os.getenv("CONFIG_FILE_PATH")

with open(CONFIG_FILE_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

**set the experience id** 

In [ ]:
EXPERIENCE_ID = config["experiments_specifique_params"]["experiment_id"]

print(EXPERIENCE_ID)

**load the metrics files**

In [ ]:
METRIC_QUERY_FILE = config["output_query_metrics_file"].format(
    experiment_id=EXPERIENCE_ID 
)

print(METRIC_QUERY_FILE)

with open(METRIC_QUERY_FILE, "r", encoding="utf-8") as f:
    metrics_json = json.load(f)

**define a function that return dict of metrcis**

In [ ]:
def format_metrics(metrics_json: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Reformats the nested metrics JSON into a flat list of dictionaries, 
    making it easier to process and plot.

    Args:
        metrics_json: The loaded JSON object (dictionary) containing 
                      metrics keyed by 'k' value as a string.

    Returns:
        A list of dictionaries, where each dictionary represents 
        a single (k, config, metric) data point.
    """
    reformatted_data = []
    
    # Iterate through the top-level keys, which are the k values (as strings)
    for k_str, config_data in metrics_json.items():
        try:
            k = int(k_str)  # Convert the k value back to an integer
        except ValueError:
            print(f"Skipping key that cannot be converted to integer k: {k_str}")
            continue
        
        # Iterate through the config/model names within that k
        for config_name, metrics in config_data.items():
            
            # Create a dictionary with only the desired fields
            print(metrics)
            data_point = {
                "k": k,
                "config": config_name,
                "precision": metrics.get("mean_precision_at_k", 0.0),
                "recall": metrics.get("mean_recall_at_k", 0.0),
                "hit_rate": metrics.get("mean_hit_rate_at_k", 0.0)
            }
            reformatted_data.append(data_point)
            
    return reformatted_data

In [ ]:
metrics = format_metrics(metrics_json)
print(metrics)

**load the folder to store the images**

In [ ]:
OUTPUT_METRIC_PLOTS_DIR = config["output_retrival_plot_metrics_dir"].format(
    experiment_id=EXPERIENCE_ID
)

**define a function to plot the metrics**

In [ ]:
def plot_metrics(reformatted_data: List[Dict[str, Any]], output_dir: str = 'model_comparisons'):
    """
    Generates a separate plot for each unique Config ID, showing all model 
    performances within that configuration, organizing plots into subfolders 
    by metric type (precision/recall).

    Args:
        reformatted_data: The flat list of dictionaries with k, config, precision, and recall.
        output_dir: The base directory where the plots will be saved. Created if it doesn't exist.
    """
    df = pd.DataFrame(reformatted_data)

    # Create the base output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving plots to base directory: {output_dir}")
    
    # --- 1. Label and Group Extraction ---
    def parse_config(config_str):
        parts = config_str.split('/')
        model_name = parts[1]
        full_config_part = parts[-1]
        return model_name, full_config_part

    df[['model_name', 'full_config_id']] = df['config'].apply(
        lambda x: pd.Series(parse_config(x))
    )
    
    unique_config_ids = df['full_config_id'].unique()
    unique_models = df['model_name'].unique()
    
    # --- 2. Color Mapping ---
    model_colors = plt.cm.get_cmap('tab10', len(unique_models))
    model_color_map = {model: model_colors(i) for i, model in enumerate(unique_models)}

    # --- 3. Plotting Loop (One plot per Config ID) ---
    for config_id in sorted(unique_config_ids): 
    
        config_data_subset = df[df['full_config_id'] == config_id].copy() 
        
        for metric_type in ['precision', 'recall', 'hit_rate']:
            
            # --- CREATE METRIC SUBFOLDER HERE ---
            metric_dir = os.path.join(output_dir, metric_type)
            os.makedirs(metric_dir, exist_ok=True)
            
            fig, ax = plt.subplots(figsize=(8, 6))
            
            for model in unique_models:
                model_data = config_data_subset[config_data_subset['model_name'] == model]
                
                if not model_data.empty:
                    ax.plot(
                        model_data['k'], 
                        model_data[metric_type], 
                        marker='o', 
                        linestyle='-', 
                        label=model,
                        color=model_color_map[model],
                        linewidth=2
                    )
            
            metric_title = metric_type.capitalize()
            
            # --- Set Plot Titles and Labels ---
            ax.set_title(f'Model Comparison for {config_id}: Mean {metric_title}@k', fontsize=14)
            ax.set_xlabel('k (Number of Retrieved Items)', fontsize=12)
            ax.set_ylabel(f'Mean {metric_title}@k', fontsize=12)
            ax.set_ylim(-0.05, 1.05)
            
            ax.legend(
                title='Model', 
                loc='upper right', 
                fontsize='small'
            ) 
            ax.grid(True, linestyle='--', alpha=0.6)
            
            plt.tight_layout()
            
            # Save the file into the metric-specific subdirectory
            filename = f"vs_k_{config_id}_comparison.png" # Removed metric_type from filename
            full_path = os.path.join(metric_dir, filename)
            plt.savefig(full_path)
            plt.close(fig)
            print(f"Generated plot: {full_path}")

In [ ]:
plot_metrics(metrics, output_dir=OUTPUT_METRIC_PLOTS_DIR)